In [167]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np


In [168]:
df = pd.read_csv("../data/CleanedFlights.csv")
df = df[df["CANCELLED"] == 0]
dfAirports = pd.read_csv("../data/Airports.csv")
pd.set_option("display.max_columns", None)

dfAirlines = pd.read_csv("../data/airlines.csv")
# creating dictionaries from the IATA_CODE to the Airport, Latitude and Longiture

AirportDict = pd.Series(dfAirports.AIRPORT.values, index = dfAirports.IATA_CODE).to_dict()
AirLineDict = pd.Series(dfAirlines.AIRLINE.values, index = dfAirlines.IATA_CODE).to_dict()
LatitudeDict = pd.Series(dfAirports.LATITUDE.values, index = dfAirports.IATA_CODE).to_dict()
LongitudeDict = pd.Series(dfAirports.LONGITUDE.values, index = dfAirports.IATA_CODE).to_dict()

df.head()

C:\Users\Florian\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning:

Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.



,Unnamed: 0,FULL_DATE,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,CANCELLED
0,0,2015-01-01,2015,1,1,Thursday,AS,ANC,SEA,00:05,23:54.0,-11.0,205.0,194.0,169.0,1448,430,408.0,-22.0,0
1,1,2015-01-01,2015,1,1,Thursday,AA,LAX,PBI,00:10,00:02.0,-8.0,280.0,279.0,263.0,2330,750,741.0,-9.0,0
2,2,2015-01-01,2015,1,1,Thursday,US,SFO,CLT,00:20,00:18.0,-2.0,286.0,293.0,266.0,2296,806,811.0,5.0,0
3,3,2015-01-01,2015,1,1,Thursday,AA,LAX,MIA,00:20,00:15.0,-5.0,285.0,281.0,258.0,2342,805,756.0,-9.0,0
4,4,2015-01-01,2015,1,1,Thursday,AS,SEA,ANC,00:25,00:24.0,-1.0,235.0,215.0,199.0,1448,320,259.0,-21.0,0


In [169]:
#  df for delay by airport and total flights from this airport

count = pd.DataFrame(df.groupby(["ORIGIN_AIRPORT"]).count()["YEAR"])
df2 = df.groupby(["ORIGIN_AIRPORT"]).mean()
preparedDfForDelayByAirport = pd.DataFrame({
    "TOTAL_FLIGHTS" : count["YEAR"], 
    "AVERAGE_ARRIVAL_DELAY": df2.ARRIVAL_DELAY })
preparedDfForDelayByAirport = preparedDfForDelayByAirport.reset_index(drop= False)
preparedDfForDelayByAirport = preparedDfForDelayByAirport.sort_values(by = "AVERAGE_ARRIVAL_DELAY")
preparedDfForDelayByAirport = preparedDfForDelayByAirport[preparedDfForDelayByAirport["TOTAL_FLIGHTS"] > 5000]
preparedDfForDelayByAirport.insert(column = "AIRPORT", loc = 0, value = preparedDfForDelayByAirport.ORIGIN_AIRPORT.map(AirportDict))
preparedDfForDelayByAirport.drop(columns = ["ORIGIN_AIRPORT"])
preparedDfForDelayByAirport = preparedDfForDelayByAirport.dropna()
# preparedDfForDelayByAirport.head()

In [170]:
# df for delay by airline
df = pd.read_csv("../data/CleanedFlights.csv")
df = df[df["CANCELLED"] == 0]

count = pd.DataFrame(df.groupby(["AIRLINE"]).count()["YEAR"])
df = df.groupby(["AIRLINE"]).mean()
dfDelayByAirline = pd.DataFrame({
    "TOTAL_FLIGHTS" : count["YEAR"], 
    "AVERAGE_ARRIVAL_DELAY": df.ARRIVAL_DELAY }) 
dfDelayByAirline = dfDelayByAirline.reset_index()
dfDelayByAirline["AIRLINE"] = dfDelayByAirline.AIRLINE.map(AirLineDict)
dfDelayByAirline = dfDelayByAirline.sort_values(by = ["AVERAGE_ARRIVAL_DELAY"])
dfDelayByAirline.head()

C:\Users\Florian\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning:

Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.



,AIRLINE,TOTAL_FLIGHTS,AVERAGE_ARRIVAL_DELAY
1,Alaska Airlines Inc.,171852,-0.976563
3,Delta Air Lines Inc.,872057,0.186754
6,Hawaiian Airlines Inc.,76101,2.023093
0,American Airlines Inc.,715065,3.451372
11,US Airways Inc.,194648,3.706209


In [171]:
# df for delay by weekday 

df = pd.read_csv("../data/CleanedFlights.csv")
df = df[df["CANCELLED"] == 0]

count = pd.DataFrame(df.groupby(["DAY_OF_WEEK"]).count()["YEAR"])
df = df.groupby(["DAY_OF_WEEK"]).mean()
dfDelayByWeekday = pd.DataFrame({
    "TOTAL_FLIGHTS" : count["YEAR"], 
    "AVERAGE_ARRIVAL_DELAY": df.ARRIVAL_DELAY }) 
dfDelayByWeekday= dfDelayByWeekday.reset_index()

# dfDelayByWeekday.head(8)

C:\Users\Florian\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning:

Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.



In [172]:
# df for delay by month

df = pd.read_csv("../data/CleanedFlights.csv")

days = [1,2,3,4,5,6,7,8,9,10,11,12]
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
monthDict = dict(zip(days, months))

count = pd.DataFrame(df.groupby(["MONTH"]).count()["YEAR"])
df = df.groupby(["MONTH"]).mean()
dfDelayByMonth = pd.DataFrame({
    "TOTAL_FLIGHTS" : count["YEAR"], 
    "AVERAGE_ARRIVAL_DELAY": df.ARRIVAL_DELAY }) 
dfDelayByMonth= dfDelayByMonth.reset_index()
dfDelayByMonth["MONTH"] = dfDelayByMonth["MONTH"].map(monthDict)

dfDelayByMonth.head(12)

,MONTH,TOTAL_FLIGHTS,AVERAGE_ARRIVAL_DELAY
0,January,469968,5.813583
1,February,429191,8.320500
2,March,504312,4.920673
3,April,485151,3.163190
4,May,496993,4.485019
5,June,503897,9.601590
6,July,520718,6.431775
7,August,510536,4.607372
8,September,464946,-0.772571
9,October,486165,-0.780390


In [185]:
# df for delay by route

df = pd.read_csv("../data/CleanedFlights.csv")
df = df[df["CANCELLED"] == 0]


dfCount = df.groupby(["ORIGIN_AIRPORT", "DESTINATION_AIRPORT"]).count().sort_values("YEAR", ascending = False)
dfCount = dfCount.reset_index()
dfCount.insert(column = "ORIGIN+DESTINATION", loc = 0, value = dfCount.ORIGIN_AIRPORT + dfCount.DESTINATION_AIRPORT)
RouteCountDict = pd.Series(dfCount.YEAR.values, index = dfCount["ORIGIN+DESTINATION"]).to_dict()

df = df.groupby(["ORIGIN_AIRPORT", "DESTINATION_AIRPORT"]).mean()
df = df.reset_index()
dfDelayByRoute = pd.DataFrame({
    "ORIGIN_AIRPORT" : df.ORIGIN_AIRPORT,
    "DESTINATION_AIRPORT" : df.DESTINATION_AIRPORT,
    "AVERAGE_ARRIVAL_DELAY": df.ARRIVAL_DELAY,
    "ORIGIN+DESTINATION": df.ORIGIN_AIRPORT + df.DESTINATION_AIRPORT,
})
dfDelayByRoute.insert(column = "TOTAL_FLIGHTS", loc = 3, value = dfDelayByRoute["ORIGIN+DESTINATION"].map(RouteCountDict))

dfDelayByRoute.drop("ORIGIN+DESTINATION", axis = 1, inplace= True)
dfDelayByRoute.insert(column = "ORIGIN_AIRPORT_LONG", loc =1, value = dfDelayByRoute["ORIGIN_AIRPORT"].map(AirportDict))
dfDelayByRoute.insert(column = "DESTINATION_AIRPORT_LONG", loc = 3, value = dfDelayByRoute["DESTINATION_AIRPORT"].map(AirportDict))
dfDelayByRoute.insert(column = "ORIGIN_AND_DESTINATION", loc = 5, value = "From " + dfDelayByRoute.ORIGIN_AIRPORT_LONG + "<br>To " + dfDelayByRoute.DESTINATION_AIRPORT_LONG )
dfDelayByRoute.head(5)

dfDelayByRoute = dfDelayByRoute.sort_values(by = ["AVERAGE_ARRIVAL_DELAY"], ascending = False)
dfDelayByRoute = dfDelayByRoute[dfDelayByRoute["TOTAL_FLIGHTS"] > 100]


C:\Users\Florian\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning:

Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.



In [197]:
fig = go.Figure()

customDelayByAirlineBest = dfDelayByAirline.drop(dfDelayByAirline.index[10:])
customDelayByAirlineWorst = dfDelayByAirline.drop(dfDelayByAirline.index[:len(dfDelayByRoute) - 10])

customDelayByRouteBest = dfDelayByRoute.drop(dfDelayByRoute.index[10:])
customDelayByRouteWorst = dfDelayByRoute.drop(dfDelayByRoute.index[:len(dfDelayByRoute) - 10])

delayByAirlineBestTenBar = go.Bar(y = customDelayByAirlineBest.AVERAGE_ARRIVAL_DELAY, name = "Delay by Airline - best ten Airlines", visible = "legendonly", showlegend= True,
                            customdata = customDelayByAirlineBest.AIRLINE,
                            hovertemplate = "%{customdata} <br>Average delay: %{y} min")

delayByAirlineWorstTenBar = go.Bar(y = dfDelayByAirline.AVERAGE_ARRIVAL_DELAY.tail(10), name = "Delay by Airline - worst ten Airlines", visible = "legendonly", showlegend= True,
                            customdata = dfDelayByAirline.AIRLINE,
                            hovertemplate = "%{customdata} <br>Average delay: %{y} min")   

delayByWeekdayBar = go.Bar(y = dfDelayByWeekday.AVERAGE_ARRIVAL_DELAY, name = "Delay by Weekday", visible = "legendonly", showlegend= True,
                        customdata = dfDelayByWeekday.DAY_OF_WEEK,
                        hovertemplate = "%{customdata} <br>Average delay: %{y} min")

delayByMonthBar = go.Bar(y = dfDelayByMonth.AVERAGE_ARRIVAL_DELAY, name = "Delay by Month", visible = "legendonly", showlegend = True,
                        customdata = dfDelayByMonth.MONTH,
                        hovertemplate = "%{customdata} <br>Average delay: %{y} min"
)
delayByRouteBestTenBar = go.Bar(y = customDelayByRouteBest.AVERAGE_ARRIVAL_DELAY, name = "Delay by Route - Best ten Routes", visible= "legendonly", showlegend = True,
                                customdata = customDelayByRouteBest.ORIGIN_AND_DESTINATION,
                                hovertemplate = "%{customdata}<br>Average delay: %{y} min")

delayByRouteWorstTenBar = go.Bar(y = customDelayByRouteWorst.AVERAGE_ARRIVAL_DELAY, name = "Delay by Route - Worst ten Routes", visible= "legendonly", showlegend = True,
                                customdata = customDelayByRouteWorst.ORIGIN_AND_DESTINATION,
                                hovertemplate = "%{customdata}<br>Average delay: %{y} min")

fig.add_trace(delayByAirlineBestTenBar) 
fig.add_trace(delayByAirlineWorstTenBar)
fig.add_trace(delayByWeekdayBar)
fig.add_trace(delayByMonthBar)
fig.add_trace(delayByRouteBestTenBar)
fig.add_trace(delayByRouteWorstTenBar)
fig.show()

